In [1]:
#Import lines
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib qt

#    The goals / steps of this project are the following:
#    1. Compute the camera calibration matrix and distortion coefficients given
#        a set of chessboard images.
#    2. Apply a distortion correction to raw images.
#    3. Use color transforms, gradients, etc., to create a thresholded binary image.
#    4. Apply a perspective transform to rectify binary image ("birds-eye view").
#    5. Detect lane pixels and fit to find the lane boundary.
#    6. Determine the curvature of the lane and vehicle position with respect to center.
#    7. Warp the detected lane boundaries back onto the original image.
#    8. Output visual display of the lane boundaries and numerical estimation
#        of lane curvature and vehicle position.

In [2]:
# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = []
        #average x values of the fitted line over the last n iterations
        self.bestx = []
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = []
        #radius of curvature of the line in some units
        self.curvature = None 
        #last curvature value
        self.curvature_last = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None
        
    def reset(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = []
        self.bestx = []
        #average x values of the fitted line over the last n iterations
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]
        self.best_fit = []
        #polynomial coefficients averaged over the last n iterations
        #radius of curvature of the line in some units
        self.curvature = None 
        #last curvature value
        self.curvature_last = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None

In [4]:
#Global method definitions

def calibrate_camera(img_path, nx=9,ny=6):
    images = glob.glob(img_path)
    imgpoints = []  #2D points in image space
    objpoints = []  #3D points in real world space
    objp = np.zeros((nx*ny,3), np.float32)
    objp[:,:2] = np.mgrid[0:nx, 0:ny].T.reshape(-1,2) #Generate x,y coordinates of image corners
    for fname in images:
        img = cv2.imread(fname)

        #convert image to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

        #find chessboard corners
        ret, corners = cv2.findChessboardCorners(gray,(nx,ny),None)

        #append points and calculate coefficients and image matrix
        if ret == True:
            imgpoints.append(corners)
            objpoints.append(objp)

    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    return mtx, dist

def undistort(img, mtx, dist):
    undst = cv2.undistort(img, mtx, dist, None, mtx)
    return undst

def perspective_transform(img, mtx, dist):
    undst = cv2.undistort(img, mtx, dist, None, mtx)
    gray = cv2.cvtColor(undst, cv2.COLOR_RGB2GRAY)
    
    img_size = (gray.shape[1], gray.shape[0])

    bot = .76
    mid = .08
    height = .62
    bot_trim = .935
    offset = img_size[0]*.25

    src = np.float32([[img.shape[1]*(.5 - mid/2), img.shape[0]*height], [img.shape[1]*(.5 + mid/2), img.shape[0]*height],\
                      [img.shape[1]*(.5 + bot/2), img.shape[0]*bot_trim], [img.shape[1]*(.5 - bot/2), img.shape[0]*bot_trim]])
    dst = np.float32([[offset, 0], [img_size[0] - offset, 0], [img_size[0] - offset, img_size[1]], [offset, img_size[1]]])
    
    # get the transform matrix and warp image to a top-down view
    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)
    warped = cv2.warpPerspective(undst, M, img_size)
    return warped, M, Minv

def abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(0, 255)):
    # Grayscale
    # Apply cv2.Sobel()
    # Create binary_output
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Apply x or y gradient with the OpenCV Sobel() function and take the absolute value
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel))
    # Rescale back to 8 bit integer
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # Create a copy and apply the threshold
    grad_binary = np.zeros_like(scaled_sobel)
    grad_binary[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    
    return grad_binary

def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    # Apply the following steps to img
    # 1) Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # 2) Take the gradient in x and y separately
    sobelx = cv2.Sobel(gray, cv2.CV_64F,1,0,ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F,0,1,ksize=sobel_kernel)
    # 3) Calculate the magnitude 
    sobel_mag = np.sqrt(np.square(sobelx)+np.square(sobely))
    # 4) Scale to 8-bit (0 - 255) and convert to type = np.uint8
    sobel_scale = np.uint8(255*sobel_mag/np.max(sobel_mag))
    # 5) Create a binary mask where mag thresholds are met
    mag_binary = np.zeros_like(sobel_scale)
    mag_binary[(sobel_scale >= mag_thresh[0])&(sobel_scale<=mag_thresh[1])] = 1
    # 6) Return this mask as your output image

    return mag_binary

def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Apply the following steps to img
    # 1) Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # 2) Take the gradient in x and y separately
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # 3) Take the absolute value of the x and y gradients
    # 4) Use np.arctan2(abs_sobely, abs_sobelx) to calculate the direction of the gradient
    gradient_dir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    # 5) Create a binary mask where direction thresholds are met
    dir_binary = np.zeros_like(gradient_dir)
    dir_binary[(gradient_dir >= thresh[0])&(gradient_dir<=thresh[1])]=1
    # 6) Return this mask as your binary_output image

    return dir_binary

def hls_select(img, thresh=(0, 255), channel=2):
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,channel]
    binary_output = np.zeros_like(s_channel)
    binary_output[(s_channel > thresh[0]) & (s_channel <= thresh[1])] = 1
    return binary_output

def yuv_select(img, thresh=(0, 255), channel=2):
    yuv = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
    y_channel = yuv[:,:,channel]
    binary_output = np.zeros_like(y_channel)
    binary_output[(y_channel > thresh[0]) & (y_channel <= thresh[1])] = 1
    return binary_output

def rgb_select(img, thresh=(0, 255), channel=2):
    rgb = img
    r_channel = rgb[:,:,channel]
    binary_output = np.zeros_like(r_channel)
    binary_output[(r_channel > thresh[0]) & (r_channel <= thresh[1])] = 1
    return binary_output

def hsv_select(img, thresh=(0, 255), channel=2):
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    h_channel = hsv[:,:,channel]
    binary_output = np.zeros_like(h_channel)
    binary_output[(h_channel > thresh[0]) & (h_channel <= thresh[1])] = 1
    return binary_output

def ycbcr_select(img, thresh=(0, 255), channel=0):

    ycrcb = cv2.cvtColor(img, cv2.COLOR_RGB2YCR_CB)
    ch = ycrcb[:,:,channel]
    ycrcb_binary = np.zeros_like(ch)
    ycrcb_binary[(ch >= thresh[0]) & (ch <= thresh[1])] = 1
    return ycrcb_binary

def lab_select(img, channel, thresh=(0, 255)):

    lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
    ch = lab[:,:,channel]
    lab_binary = np.zeros_like(ch)
    lab_binary[(ch >= thresh[0]) & (ch <= thresh[1])] = 1
    return lab_binary

def threshold(image):
    # Choose a Sobel kernel size
    ksize = 9 # Choose a larger odd number to smooth gradient measurements

    # Apply each of the thresholding functions
    gradx = abs_sobel_thresh(image, orient='x', sobel_kernel=ksize, thresh=(40, 120))
    grady = abs_sobel_thresh(image, orient='y', sobel_kernel=ksize, thresh=(20, 100))
    mag_binary = mag_thresh(image, sobel_kernel=ksize, mag_thresh=(50, 90))
    dir_binary = dir_threshold(image, sobel_kernel=ksize, thresh=(0.9, 1.3))
    s_binary = hls_select(image, thresh=(140, 255))
    y_binary = yuv_select(image, thresh=(0, 112), channel=1) #yellow lines
    h_binary = hsv_select(image, thresh=(18, 115), channel=2) #shadows
    u_binary = yuv_select(image, thresh=(200, 255), channel=0) #shadows
    b_binary = hls_select(image, thresh=(150, 255), channel=1)
    r_binary = rgb_select(image, thresh=(195, 255), channel=0)
    g_binary = rgb_select(image, thresh=(210, 255), channel=1)
    v_binary = yuv_select(image, thresh=(0, 95), channel=1)
    ycbcr_binary = ycbcr_select(image, thresh=(210,255), channel=0)
    lab_binary = lab_select(image, thresh=(150,255), channel=2)
    
    #combine thresholds to create a single binary image
    combined = np.zeros_like(dir_binary)
    combined[((gradx == 1) & (grady == 1)) |
             ((mag_binary == 1) & (dir_binary == 1)) |
             ((v_binary == 1)&(s_binary == 1)) |
             #((y_binary == 1)&(s_binary == 1)) |
             #((b_binary == 1)&(y_binary == 1))|
             (r_binary == 1) |
             (g_binary == 1)| (lab_binary == 1) | (ycbcr_binary == 1)
            ] = 1  
    
    return combined

In [5]:
def sliding_window_fit(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]/2:,:], axis=0)

    # Create an output image to draw on and  visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))#*255
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Choose the number of sliding windows
    nwindows =9
    # Set height of windows
    window_height = np.int(binary_warped.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 105
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,1,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,1,0), 2) 
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    #Use this section to output binary image with detected lanes
    #--------------------------------------------------------------------------
    #out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [1, 0, 0]
    #out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 1]
    #plt.figure()
    #plt.imshow(out_img)
    #plt.plot(left_fitx, ploty, color='yellow')
    #plt.plot(right_fitx, ploty, color='yellow')
    #plt.xlim(0, 1280)
    #plt.ylim(720, 0)
    #---------------------------------------------------------------------------
    
    return ploty, left_fit, right_fit, left_fitx, right_fitx

def skip_fit(binary_warped, left_fit, right_fit):
    # Assume you now have a new warped binary image 
    # from the next frame of video (also called "binary_warped")
    # It's now much easier to find line pixels!
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    margin = 100
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] + margin))) 
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] + margin)))  

    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    return ploty, left_fit, right_fit, left_fitx, right_fitx

def curvature(ploty, left_fit, right_fit, left_fitx, right_fitx):
    y_eval = np.max(ploty)
    left_curverad = ((1 + (2*left_fit[0]*y_eval + left_fit[1])**2)**1.5) / np.absolute(2*left_fit[0])
    right_curverad = ((1 + (2*right_fit[0]*y_eval + right_fit[1])**2)**1.5) / np.absolute(2*right_fit[0])
    #print(left_curverad, right_curverad)
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(ploty*ym_per_pix, left_fitx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, right_fitx*xm_per_pix, 2)
    # Calculate the new radii of curvature
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    # Now our radius of curvature is in meters
    return left_curverad, right_curverad

def diff_change(y1, y2):

    if y1 == 0. or y2==0.:
        return 0.
    return np.abs(((y2-y1)/y1))


In [6]:
#Test Images Sandbox

#mtx, dist = calibrate_camera('.\camera_cal\calibration*.jpg')

#images = glob.glob('.\*.jpg')

#for fname in images:

#    unwarped_image = cv2.imread(fname)
#    image, M, Minv = perspective_transform(unwarped_image, mtx, dist)

#    combined = threshold(image)
    
#    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
#    f.tight_layout()
#    unwarped_image = cv2.cvtColor(unwarped_image, cv2.COLOR_BGR2RGB)
#    ax1.imshow(unwarped_image)
#    ax1.set_title('Original Image', fontsize=50)
#    ax2.imshow(combined, cmap = 'gray')
#    ax2.set_title('Undistorted and Warped  Binary Image', fontsize=24)
#    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
#    plt.tight_layout()
#    plt.show()

#    ploty, left_fit, right_fit, left_fitx, right_fitx = sliding_window_fit(combined)
#    skip_fit(combined, left_fit, right_fit)
    
#    curvature(ploty, left_fit, right_fit, left_fitx, right_fitx)


In [7]:
mtx, dist = calibrate_camera('.\camera_cal\calibration*.jpg')

#define Lane class
left = Line()
right = Line()

left_dropped = 0
right_dropped = 0


def process_image(image):
    global left
    global right
    global left_dropped
    global right_dropped
    
    original_image = image
    warped, M, Minv = perspective_transform(original_image, mtx, dist)
    
    combined = threshold(warped)
    
    #This section resets the lane lines if the number of dropped frames reaches
    #a certain limit
    if left_dropped >= 100:
        left.reset
        left_dropped = 0
    if right_dropped >= 100:
        right.reset
        right_dropped = 0
        
    #Genereate polynomial fit based on previous lane detection
    if left.detected == False or right.detected == False:
        ploty, left_fit, right_fit, left_fitx, right_fitx = sliding_window_fit(combined)
        left.detected = True
        right.detected = True
        #print('dropped')
    else:       
        #Use an abbreviated fit based on last line location if line is detected
        ploty, left_fit, right_fit, left_fitx, right_fitx = skip_fit(combined, left.current_fit[-1], right.current_fit[-1])
        left.detected = True
        right.detected = True
        #print('skipfit')
    
    #populate line class parameters
    left.recent_xfitted.append(left_fitx)
    right.recent_xfitted.append(right_fitx)
    left.current_fit.append(left_fit)
    right.current_fit.append(right_fit)
        
    
    if len(left.current_fit) <= 20:
        left.best_fit.append(np.mean(left.current_fit, axis=0))
        right.best_fit.append(np.mean(right.current_fit, axis=0))
        left.bestx.append(np.mean(left.recent_xfitted, axis=0))
        right.bestx.append(np.mean(right.recent_xfitted, axis=0))
    else:
        left.diffs = np.abs(np.divide(np.subtract(left.current_fit[-1],left.best_fit[-1]),left.best_fit[-1]))
        right.diffs = np.abs(np.divide(np.subtract(right.current_fit[-1], right.best_fit[-1]),right.best_fit[-1]))
        left.best_fit.append(np.mean(left.current_fit[-10:], axis=0))
        right.best_fit.append(np.mean(right.current_fit[-10:], axis = 0))
        left.bestx.append(np.mean(left.recent_xfitted[-10:], axis=0))
        right.bestx.append(np.mean(right.recent_xfitted[-10:], axis=0))
    
    #Calculate curvature taking into account straight lines having high curvature values
    left.curvature, right.curvature = curvature(ploty, left.best_fit[-1], right.best_fit[-1], left.bestx[-1], right.bestx[-1])
    if left.curvature > 1000.:
        left.curvature = 0.
    if right.curvature > 1000.:
        right.curvature = 0.
        
    if len(left.best_fit) <= 1 or len(right.best_fit) <= 1:
        left.curvature_last, right.curvature_last = left.curvature, right.curvature     
    else:
        left.curvature_last, right.curvature_last = curvature(ploty, left.best_fit[-2], right.best_fit[-2], left.bestx[-2], right.bestx[-2])
    
    #Tolerances for dropping lanes based on curvature, and line fit difference
    if right.curvature-left.curvature > 2.5:     
        if np.any(left.diffs<= 30.) and np.any(left.diffs >= 5.0) and np.all(left.diffs is not 0.): #or diff_change(left.curvature_last, left.curvature)>0.4:
            left.best_fit.pop()
            left.bestx.pop()
            left.detected = False
            left_dropped+=1
 
        if np.any(right.diffs<= 30.) and np.any(right.diffs >= 5.0) and np.all(right.diffs is not 0.): #or diff_change(right.curvature_last, right.curvature)>0.4:
            right.best_fit.pop()
            right.bestx.pop()
            right.detected = False
            right_dropped+=1 
    
    #diagnostics
    #---------------------------------------------------------------------------    
    #print(left.diffs)
    #print(right.diffs)
    #print(diff_change(right.curvature_last, right.curvature))
    #print(diff_change(left.curvature_last, left.curvature))
    #print(left_dropped)
    #print(right_dropped)
    #---------------------------------------------------------------------------
    
    
    if left.detected==True or right.detected == True:
        #Create an image to draw the lines on
        warp_zero = np.zeros_like(warped).astype(np.uint8)
        color_warp = warp_zero #np.dstack((warp_zero, warp_zero, warp_zero))
        
        # Recast the x and y points into usable format for cv2.fillPoly()
        pts_left = np.array([np.transpose(np.vstack([left.bestx[-1], ploty]))])
        pts_right = np.array([np.flipud(np.transpose(np.vstack([right.bestx[-1], ploty])))])
        pts = np.hstack((pts_left, pts_right))

        # Draw the lane onto the warped blank image
        cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

        # Warp the blank back to original image space using inverse perspective matrix (Minv)
        newwarp = cv2.warpPerspective(color_warp, Minv, (image.shape[1], image.shape[0])) 
        # Combine the result with the original image
        result = cv2.addWeighted(original_image, 1, newwarp, 0.3, 0)
        
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(result,'radius of curvature  = %.2f m'%((left.curvature+right.curvature)/2),(50,50), font, 1,(255,255,255),2,cv2.LINE_AA)
        is_tracking = left.detected or right.detected
        cv2.putText(result,'Tracking Locked' if is_tracking else 'Tracking Lost',(50,100), font, 1,(0,255,0) if is_tracking else (255,0,0), 3,cv2.LINE_AA)

        return result
    
#Load Video Clip
output = 'video_output\out_video.mp4'
clip = VideoFileClip(".\project_video.mp4")
drawn_clip = clip.fl_image(process_image)
%time drawn_clip.write_videofile(output, audio=False)

[MoviePy] >>>> Building video video_output\out_video.mp4
[MoviePy] Writing video video_output\out_video.mp4


100%|█████████████████████████████████████████████████████████████████████████████▉| 1260/1261 [03:58<00:00,  5.37it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: video_output\out_video.mp4 

Wall time: 3min 59s
